In [4]:
import os
import requests
import pandas as pd

In [5]:
projects = requests.get("https://dcc.icgc.org/api/v1/projects?size=1000").json()["hits"]
print "Found {} ICGC projects".format(len(projects))

data_types = ["simple_somatic_mutation.open", "exp_array", "donor"]

files = [{"name": "{1}.{0}.tsv.gz".format(p["id"], d),
          "url": "https://dcc.icgc.org/api/v1/download?fn=/release_22/Projects/{0}/{1}.{0}.tsv.gz".format(p["id"], d)}
         for p in projects for d in data_types]
files_to_download = [f for f in files if not os.path.isfile("/data/icgc/{}".format(f["name"]))]
print "Files to download", len(files_to_download)

Found 70 ICGC projects
Files to download 60


In [ ]:
for f in files_to_download:
    print("Downloading {}".format(f["name"]))
    r = requests.get(f["url"], allow_redirects=True, verify=False)
    if r.status_code == requests.codes.ok:
        with open("/data/icgc/{}".format(f["name"]), 'wb+') as tar:
            tar.write(r.content)
    else:
        print("Problems downloading {}: {}".format(f["name"], r.status_code))

In [7]:
donors = pd.read_table("/data/icgc/donor.ALL-US.tsv.gz")
donors.head()

,icgc_donor_id,project_code,study_donor_involved_in,submitted_donor_id,donor_sex,donor_vital_status,disease_status_last_followup,donor_relapse_type,donor_age_at_diagnosis,donor_age_at_enrollment,...,donor_relapse_interval,donor_diagnosis_icd10,donor_tumour_staging_system_at_diagnosis,donor_tumour_stage_at_diagnosis,donor_tumour_stage_at_diagnosis_supplemental,donor_survival_time,donor_interval_of_last_followup,prior_malignancy,cancer_type_prior_malignancy,cancer_history_first_degree_relative
0,DO2,ALL-US,NaN,TARGET-10-PAIXFI,male,NaN,NaN,NaN,5,5,...,2865,C91.0,NaN,NaN,NaN,2865,2920,NaN,NaN,NaN
1,DO1,ALL-US,NaN,TARGET-10-PAIXDK,male,deceased,relapse,progression (liquid tumours),7,7,...,1169,C91.0,NaN,NaN,NaN,2037,2037,NaN,NaN,NaN
2,DO4,ALL-US,NaN,TARGET-10-PAIXGP,female,alive,NaN,NaN,2,2,...,5087,C91.0,NaN,NaN,NaN,5087,5087,NaN,NaN,NaN
3,DO3,ALL-US,NaN,TARGET-10-PAIXFN,male,deceased,relapse,progression (liquid tumours),15,15,...,650,C91.0,NaN,NaN,NaN,1068,1068,NaN,NaN,NaN
4,DO6,ALL-US,NaN,TARGET-10-PAIXPN,male,alive,NaN,NaN,1,1,...,4946,C91.0,NaN,NaN,NaN,4946,4946,NaN,NaN,NaN


In [9]:
donors.describe()

/opt/conda/envs/python2/lib/python2.7/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


,study_donor_involved_in,donor_age_at_diagnosis,donor_age_at_enrollment,donor_age_at_last_followup,donor_relapse_interval,donor_tumour_staging_system_at_diagnosis,donor_tumour_stage_at_diagnosis,donor_tumour_stage_at_diagnosis_supplemental,donor_survival_time,donor_interval_of_last_followup,prior_malignancy,cancer_type_prior_malignancy,cancer_history_first_degree_relative
count,0.0,1002.000000,1002.000000,1002.000000,1002.000000,0.0,0.0,0.0,1002.000000,1002.000000,0.0,0.0,0.0
mean,NaN,9.927146,9.927146,16.584830,2105.264471,NaN,NaN,NaN,2435.883234,2432.849301,NaN,NaN,NaN
std,NaN,5.805857,5.805857,6.196845,1288.331783,NaN,NaN,NaN,1175.223470,1173.425799,NaN,NaN,NaN
min,NaN,1.000000,1.000000,2.000000,20.000000,NaN,NaN,NaN,28.000000,0.000000,NaN,NaN,NaN
25%,NaN,4.000000,4.000000,12.000000,905.750000,NaN,NaN,NaN,1509.000000,1460.000000,NaN,NaN,NaN
50%,NaN,11.000000,11.000000,17.000000,2281.000000,NaN,NaN,NaN,2775.000000,2751.000000,NaN,NaN,NaN
75%,NaN,15.000000,15.000000,21.000000,3154.000000,NaN,NaN,NaN,3236.500000,3253.500000,NaN,NaN,NaN
max,NaN,30.000000,30.000000,34.000000,5137.000000,NaN,NaN,NaN,5204.000000,5204.000000,NaN,NaN,NaN


In [31]:
expression = pd.read_table("/data/icgc/exp_array.ALL-US.tsv.gz")
expression.head()

,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,submitted_sample_id,analysis_id,gene_model,gene_id,normalized_expression_value,fold_change,platform,experimental_protocol,normalization_algorithm,other_analysis_algorithm,raw_data_repository,raw_data_accession,reference_sample_type
0,DO2,ALL-US,SP2,SA4,TARGET-10-PAIXFI-03A-01R,TARGET_ALL_57_20120329,RefSeq,NM_013994,1834.0310,NaN,Affymetrix Human U133 Plus 2.0,cRNA http://www.ebi.ac.uk/arrayexpress/experim...,MAS5.0,Collapsed Dataset - http://www.broadinstitute....,GEO,GSE11877,unrelated normal
1,DO2,ALL-US,SP2,SA4,TARGET-10-PAIXFI-03A-01R,TARGET_ALL_57_20120329,RefSeq,NM_181471,941.9643,NaN,Affymetrix Human U133 Plus 2.0,cRNA http://www.ebi.ac.uk/arrayexpress/experim...,MAS5.0,Collapsed Dataset - http://www.broadinstitute....,GEO,GSE11877,unrelated normal
2,DO2,ALL-US,SP2,SA4,TARGET-10-PAIXFI-03A-01R,TARGET_ALL_57_20120329,RefSeq,NM_002155,173.2684,NaN,Affymetrix Human U133 Plus 2.0,cRNA http://www.ebi.ac.uk/arrayexpress/experim...,MAS5.0,Collapsed Dataset - http://www.broadinstitute....,GEO,GSE11877,unrelated normal
3,DO2,ALL-US,SP2,SA4,TARGET-10-PAIXFI-03A-01R,TARGET_ALL_57_20120329,RefSeq,XM_005263758,931.3263,NaN,Affymetrix Human U133 Plus 2.0,cRNA http://www.ebi.ac.uk/arrayexpress/experim...,MAS5.0,Collapsed Dataset - http://www.broadinstitute....,GEO,GSE11877,unrelated normal
4,DO2,ALL-US,SP2,SA4,TARGET-10-PAIXFI-03A-01R,TARGET_ALL_57_20120329,RefSeq,NM_000409,264.4506,NaN,Affymetrix Human U133 Plus 2.0,cRNA http://www.ebi.ac.uk/arrayexpress/experim...,MAS5.0,Collapsed Dataset - http://www.broadinstitute....,GEO,GSE11877,unrelated normal


In [82]:
variants = pd.read_table("/data/icgc/simple_somatic_mutation.open.ALL-US.tsv.gz")
variants.head()

,icgc_mutation_id,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,matched_icgc_sample_id,submitted_sample_id,submitted_matched_sample_id,chromosome,chromosome_start,...,experimental_protocol,sequencing_strategy,base_calling_algorithm,alignment_algorithm,variation_calling_algorithm,other_analysis_algorithm,seq_coverage,raw_data_repository,raw_data_accession,initial_data_release_date
0,MU4819276,DO5,ALL-US,SP873,SA565,SA167,TARGET-10-PAIXPH-03A-01D,TARGET-10-PAIXPH-10A-01D,12,25271550,...,Paired End,WXS,CASAVA,BWA,SNVMix2,NaN,NaN,CGHub,27e629e7-b3c8-4d3b-8ddc-0ff89433e96d,NaN
1,MU4819276,DO5,ALL-US,SP873,SA565,SA167,TARGET-10-PAIXPH-03A-01D,TARGET-10-PAIXPH-10A-01D,12,25271550,...,Paired End,WXS,CASAVA,BWA,SNVMix2,NaN,NaN,CGHub,27e629e7-b3c8-4d3b-8ddc-0ff89433e96d,NaN
2,MU4819276,DO5,ALL-US,SP873,SA565,SA167,TARGET-10-PAIXPH-03A-01D,TARGET-10-PAIXPH-10A-01D,12,25271550,...,Paired End,WXS,CASAVA,BWA,SNVMix2,NaN,NaN,CGHub,27e629e7-b3c8-4d3b-8ddc-0ff89433e96d,NaN
3,MU4819276,DO5,ALL-US,SP873,SA565,SA167,TARGET-10-PAIXPH-03A-01D,TARGET-10-PAIXPH-10A-01D,12,25271550,...,Paired End,WXS,CASAVA,BWA,SNVMix2,NaN,NaN,CGHub,27e629e7-b3c8-4d3b-8ddc-0ff89433e96d,NaN
4,MU4819276,DO5,ALL-US,SP873,SA565,SA167,TARGET-10-PAIXPH-03A-01D,TARGET-10-PAIXPH-10A-01D,12,25271550,...,Paired End,WXS,CASAVA,BWA,SNVMix2,NaN,NaN,CGHub,27e629e7-b3c8-4d3b-8ddc-0ff89433e96d,NaN


In [91]:
variants.columns

Index([u'icgc_mutation_id', u'icgc_donor_id', u'project_code',
       u'icgc_specimen_id', u'icgc_sample_id', u'matched_icgc_sample_id',
       u'submitted_sample_id', u'submitted_matched_sample_id', u'chromosome',
       u'chromosome_start', u'chromosome_end', u'chromosome_strand',
       u'assembly_version', u'mutation_type', u'reference_genome_allele',
       u'mutated_from_allele', u'mutated_to_allele', u'quality_score',
       u'probability', u'total_read_count', u'mutant_allele_read_count',
       u'verification_status', u'verification_platform',
       u'biological_validation_status', u'biological_validation_platform',
       u'consequence_type', u'aa_mutation', u'cds_mutation', u'gene_affected',
       u'transcript_affected', u'gene_build_version', u'platform',
       u'experimental_protocol', u'sequencing_strategy',
       u'base_calling_algorithm', u'alignment_algorithm',
       u'variation_calling_algorithm', u'other_analysis_algorithm',
       u'seq_coverage', u'raw_data_re

In [97]:
len(variants.groupby("submitted_sample_id"))

34

In [89]:
# Submissions dict keyed by icgc_sample_id
submissions = {id: {} for id in expression.analyzed_sample_id.append(variants.analyzed_sample_id).unique()}
print "Found {} samples with either or both expression and variant data".format(len(submissions))
    
# for sample_id in expression.icgc_sample_id.unique():
#     submissions[sample_id]["expression"] = expression[expression["icgc_sample_id"] == sample_id][
#         ["gene_id", "normalized_expression_value"]]
#     submissions[sample_id]["icgc_donor_id"] = expression[expression["icgc_sample_id"] == sample_id].iloc[0]

# for sample_id in variants.icgc_sample_id.unique():
#     submissions[sample_id]["variants"] = variants[variants["icgc_sample_id"] == sample_id][
#         ["chromosome", "chromosome_start", "mutated_from_allele", "mutated_to_allele"]]
#     submissions[sample_id]["icgc_donor_id"] = \
#         variants[variants["icgc_sample_id"] == sample_id]["icgc_donor_id"].get_value
# submissions["SA565"]


AttributeError: 'DataFrame' object has no attribute 'analyzed_sample_id'